# 레시피 추천 시스템

https://chan-chance.tistory.com/22

In [52]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from gensim.models import Word2Vec

In [53]:
coupang = pd.read_csv('../../data/coupang_filtering.csv')
coupang.drop('Unnamed: 0', axis=1, inplace=True)

ckg = pd.read_csv('../../data/ckg_drop.csv')
ckg.drop('Unnamed: 0', axis=1, inplace=True)
ckg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97378 entries, 0 to 97377
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   레시피일련번호   97378 non-null  int64 
 1   레시피제목     97378 non-null  object
 2   요리명       95875 non-null  object
 3   조회수       97378 non-null  int64 
 4   추천수       97378 non-null  int64 
 5   스크랩수      97378 non-null  int64 
 6   요리방법별명    97378 non-null  object
 7   요리상황별명    96911 non-null  object
 8   요리재료별명    97378 non-null  object
 9   요리종류별명    97378 non-null  object
 10  요리소개      97018 non-null  object
 11  요리재료내용    97378 non-null  object
 12  요리재료_전처리  97374 non-null  object
dtypes: int64(4), object(9)
memory usage: 9.7+ MB


## 만개의 레시피에서 조회수 기반 스크랩수 가중치 계산

In [54]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating

    ( (각 조회수 / (각 조회수+최소조회수)) * 각 스크랩수 ) + ( (최소조회수/(각 조회수+최소조회수)) * 평균스크랩수 )
    
    Returns: pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [55]:
# calcuate input parameters
C = np.mean(ckg['스크랩수'])
m = np.percentile(ckg['조회수'], 70)
vote_count = ckg[ckg['조회수'] >= m]
R = ckg['스크랩수']
v = ckg['조회수']

ckg['weighted_clipping'] = weighted_rating(v,m,R,C)

#결측치 제거
ckg.dropna(inplace=True)

#word2vec 사용을 위한 데이터 포멧 변경
ckg['요리재료_전처리'] = ckg['요리재료_전처리'].apply(lambda x : x.split(', ')) # 1차원 list 형태
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
2,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]",318.067826
3,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루]",287.543703
4,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,"[파스타면, 토마토, 토마토, 양파, 다진마늘, 피망, 올리브유]",214.806809
5,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일, 녹말가루, 물, 계란 노...",280.964458
6,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,"[닭, 통마늘, 다진마늘, 화이트와인또는우유, 올리브유또는버터, 마늘솔트또는소금, ...",253.840139


# word2vect 기반
> 요리명과 상품명이 정확히 일치하지 않으면 레시피가 아예 나오지 않음

In [56]:
# 쿠팡 상품 하나 선택
product_df = coupang.iloc[30:31, :]
product_df

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
30,1,6909265763,16645168518,83829261628,바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채,"바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채, 1세트",27000,15%,22800,NaN,5.0,5,0,"아귀, 바지락, 홍합, 야채세트(콩나물, 미나리, 파, 고추, 무), 바른마늘, 해...","아귀,바지락,홍합,콩나물,미나리,파,고추,무,해물탕 양념",생선탕


In [65]:
product = product_df.상품.values.tolist()
product_list = ' '.join(s for s in product)
product_list = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』]', '', product_list).split(' ') #특수문자제거
product_list = list(filter(None, product_list)) #null 제거
product_list

['바다자리', '생물', '아귀', '매운탕', '씨키트', '23인분', '1kg', '야채']

In [66]:
recipe_df = pd.DataFrame()
for word in product_list:
  #recipe = ckg[ckg['요리명']==word]
  recipe = ckg[ckg['요리명'].str.contains(word)]
  recipe_df = pd.concat([recipe_df, recipe])
recipe_df

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
76458,6924171,겨울 바다 내음을 먹고 싶다면 생물미역 무침,생물미역무침,7746,0,88,무침,일상,해물류,밑반찬,생물미역을 데치지 않고 싱싱한 미역 그대로 생으로 무쳐 봤습니다. 청양고추와 레몬...,[재료] 물미역 1묶음| 양파 1/4개| 대파 1/2대| 청양고추 2개 [양념] 간...,"[물미역, 양파, 대파, 청양고추, 간장, 다진마늘, 참치액, 매실청, 고추가루, ...",276.291844
77329,6925361,새콤달콤하게~꼬득꼬득한 생물미역대무침!,생물미역대무침,7697,0,118,무침,일상,해물류,밑반찬,생물미역줄기만 따로 팔기도하지요~물미역은 부드러운 식감이라면 미역줄기(대)는 꼬득꼬...,[재료] 미역줄기 500g [양념] 고추가루 2큰술| 사과식초 2~3큰술| 다진마늘...,"[미역줄기, 고추가루, 사과식초, 다진마늘, 설탕, 소금, 통깨]",287.691079
82270,6932221,제철요리 1마리에 천원! 봄에만 나오는 너무 맛있는 국산 생물꽁치조림,생물꽁치조림,450,0,12,조림,초스피드,기타,메인반찬,저는 장을 보러가면 수산시장부터 들러 생선들을 사죠 주로 제철생선 위주로 사는편이에...,[재료] 생물 꽁치 4마리| 무 조금 [양념장] 물 1컵반| 굴소스 1T| 간장 2...,"[생물 꽁치, 무, 물, 굴소스, 간장, 맛술, 고추가루, 마늘, 꿀, 올리브오일, 파]",373.229197
237,734622,매콤해서 더 맛있는 아귀찜,아귀찜,109727,32,1106,찜,손님접대,해물류,메인반찬,아구찜을 밖에서 사먹으면 아구는 안보이고 콩나물이랑 야채만 먹게되는데 집에서 만들어...,[재료] 아귀 1~1.5kg| 콩나물 800g| 미나리(or깻잎/고추) 200g| ...,"[아귀, 콩나물, 미나리, 대파, 콩나물 삶은국물, 찹쌀가루, 참기름, 통깨, 소금...",1027.737021
1837,1617475,아귀튀김 강정,아귀튀김강정,5377,10,23,튀김,손님접대,해물류,밑반찬,아귀튀김 간정 만들었어요~ 바삭바삭하면서도 살이 푸짐하고 부드러워서 밥도둑이 될만한...,[재료] 아귀 500g| 전분 1/2컵| 튀김기름 적당량 [양념] 고추장 1큰술| ...,"[아귀, 전분, 튀김기름, 고추장, 두반장, 설탕, 매실액, 고추가루, 간장, 다진...",281.442723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96616,6951401,새콤달콤한 야채 비빔만두 레시피,야채비빔만두,772,0,11,비빔,간식,가공식품류,면/만두,냉동실에 혹시 냉동 만두가 굴러다니고 있지는 않나요? 오늘은 냉동실에 잠자고 있는 ...,[재료] 냉동만두 1봉지| 집에 있는 야채들 약간| 통깨 약간| 식용유 약간 [양념...,"[냉동만두, 집에 있는 야채들, 통깨, 식용유, 고추장, 식초, 설탕, 다진마늘, ...",364.949175
96637,6951434,어른 아이반찬 소세지야채볶음레시피,소세지야채볶음,5583,0,161,볶음,일상,가공식품류,밑반찬,남녀노소 누구나 좋아하는 소세지로 반찬을 만들어봤어요.,[재료] 비엔나소세지 300g| 양파 1/3개| 파프리카 1/2개| 페퍼론치노 5개...,"[비엔나소세지, 양파, 파프리카, 페퍼론치노, 식용유, 케찹, 고추장, 굴소스, 올...",319.286475
96673,6951482,소고기야채쌈말이,소고기야채쌈말이,3186,0,48,볶음,손님접대,소고기,메인반찬,내인생의 처음으로 해보는 요리중 하나인 소고기야채쌈말이 입니다.,[식재료] 소고기 300g| 팽이버섯 1팩| 쪽파 10개| 당근 1/4개| 노란피망...,"[소고기, 팽이버섯, 쪽파, 당근, 노란피망, 간장, 설탕, 후추약간, 올리고당, ...",320.458005
96957,6951881,새우카레야채볶음밥,새우카레야채볶음밥,663,0,21,볶음,일상,채소류,밥/죽/떡,냉장고속 조각조각 야채도 정리할 겸 야채볶음밥을 만들어 먹었어요~ 이왕이면 좀더 이...,[재료] 새우 6마리| 양파 1/3개| (노랑)파프리카 1/4개| (빨강)파프리카 ...,"[새우, 양파, 파프리카, 파프리카, 호박, 당근, 쪽파, 밥, 버터, 파슬리가루,...",368.171129


In [67]:
str_list = product_df.구성정보_전처리.values.tolist()
ingredient = ' '.join(s for s in str_list).split(',')
print(ingredient)

word = [words for words in recipe_df.요리재료_전처리]
word.append(ingredient)
embedding_model = Word2Vec(sentences=word, vector_size=100, window=5, min_count=1,  sg=1, batch_words=1000)

['아귀', '바지락', '홍합', '콩나물', '미나리', '파', '고추', '무', '해물탕 양념']


In [68]:
# 단어 벡터 평균구하기
def vectors(embedding):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(embedding_model.wv[word]) 
            count+=1
        except : continue

    word_embedding2=np.array(word_embedding)
    v=word_embedding2.sum(axis=0)  
    
    if type(v)!=np.ndarray:
        return np.zeros(100)
    
    return(v/count)

#최종 알고리즘
def wv_mealkit(mealkit_index,sim,top_n):
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]

    # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
    similar_indexes=sim_sorted_ind[mealkit_index,1:(top_n+1)]
    recipe = recipe_df.iloc[similar_indexes].sort_values(by='weighted_clipping',ascending=False)
    
    return recipe

In [69]:
wordvec=[vectors(x) for x in word]

# 1. 코사인유사도
w2v_sim=cosine_similarity(wordvec,wordvec)
# 2. 유클리디언 유사도
w2v_euc=euclidean_distances(wordvec,wordvec)
# 3. 맨하탄 유사도 
w2v_man=manhattan_distances(wordvec,wordvec)

1. 코사인유사도

In [70]:
#1. 코사인유사도
mealkit_index = len(w2v_sim)-1

sim_mealkit_df = wv_mealkit(mealkit_index, w2v_sim, 5)
sim_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
39676,6868917,아귀맑은탕,아귀탕,39662,4,995,끓이기,손님접대,해물류,국/탕,시원한 국물맛이 일품 아귀맑은탕 끓였어요 국물 시원한 것이 완전 해장국 입니다,[재료] 아귀 500g| 무 1/5개| 콩나물 100g| 미나리 100g| 대파 1...,"[아귀, 무, 콩나물, 미나리, 대파, 청양고추, 까나리액젓, 다진마늘]",841.318178
95172,6949448,탕인 아귀와 야채를 활용해서 만든 [아귀탕 지리] 입니다.,아귀탕지리,2182,0,19,끓이기,일상,해물류,국/탕,아귀와 야채로 만들어 본 시원한 탕인 아귀탕 지리 만들어 봤어요~!!^^*,[재료] 무 4/1개| 파 2개| 미나리 1/2단| 콩나물 1/2단| 참치액젓 2큰...,"[무, 파, 미나리, 콩나물, 참치액젓, 아귀]",333.979312
66342,6909895,시원한 국물 아귀탕 쉬운 레시피,아귀탕,31569,0,294,끓이기,손님접대,해물류,국/탕,아귀탕 어렵게 생각하지마시고 도전해보세요,[재료] 아귀 1마리| 무 1줌| 콩나물 1줌| 미나리 1줌 [양념] 소금 1T| ...,"[아귀, 무, 콩나물, 미나리, 소금, 다진마늘]",321.187865
40013,6869415,저녁메뉴~~~못난이 아귀의 맛난변신 얼큰시원한 국물이 일품인 아귀맑은탕,아귀탕,9858,4,74,끓이기,일상,해물류,국/탕,뜨건국물에 미세먼지가 씻겨나가길 바라며 시원한 아귀탕을 끓여봤어요,[재료] 아귀 1마리| 표고버섯 5~6개| 콩나물 1줌| 무 약간| 대파 1뿌리| ...,"[아귀, 표고버섯, 콩나물, 무, 대파, 청양고추, 다진마늘, 소금, 멸치육수, 미나리]",253.228840
32875,6857073,회 뜨고 남은 생선 매운탕,생선매운탕,49240,7,136,끓이기,기타,해물류,국/탕,안녕하세요. 초이맘이예요~~^^ 회 뜨고 나서 매운탕 거리를 챙겨 주시더라고요 ...,[재료] 매운탕생선| 무 1토막| 콩나물 150g| 깻잎 20장| 미나리 1줌| 대...,"[매운탕생선, 무, 콩나물, 깻잎, 미나리, 대파, 다진마늘, 청량고추, 떡볶이떡,...",189.252253


In [71]:
ingredient = set(ingredient)
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in sim_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  {'콩나물', '고추', '홍합', '미나리', '무', '해물탕 양념', '파', '바지락', '아귀'}


[['무', '미나리', '대파', '청양고추', '까나리액젓', '다진마늘'],
 ['파', '콩나물', '참치액젓'],
 ['무', '미나리', '소금', '다진마늘'],
 ['표고버섯', '무', '대파', '청양고추', '다진마늘', '소금', '멸치육수'],
 ['매운탕생선', '콩나물', '깻잎', '대파', '다진마늘', '청량고추', '떡볶이떡', '양념소스']]

2. 유클리디언 유사도

In [72]:
#2. 유클리디언 유사도
mealkit_index = len(w2v_euc)-1

euc_mealkit_df = wv_mealkit(mealkit_index, w2v_euc, 5)
euc_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
29687,6851213,[해외자취Cook.feel通]157. 치즈야채수제비 레시피,치즈야채수제비,450,0,3,끓이기,초스피드,밀가루,면/만두,전날 한 야채수제비가 2인분이라 수제비 반죽을 뜯어 넣기 전에 1인분을 따로 덜어놨...,[재료] 먹다남은 야채수제비육수 2국자| 수제비반죽 1주먹| 체다치즈 1장,"[먹다남은 야채수제비육수, 수제비반죽, 체다치즈]",372.936123
38594,6867336,야채가 들어간 비빔면,야채비빔면,704,2,5,비빔,초스피드,채소류,면/만두,봄이 오니까 자꾸 시원한게 생각이 나시죠? 저두요. . . 그래서 비빔면 삶아 샐러...,[재료] 비빔면 1봉| 새러드 채소 약간| 토마토 1/6개,"[비빔면, 새러드 채소, 토마토]",366.359289
95027,6949241,[친환경자연쿡]현미라이스페이퍼야채찜,현미라이스페이퍼야채찜,790,0,9,찜,영양식,기타,메인반찬,현미라이스페이퍼로 두부와 야채를 잘게 다져 돌돌 말아 쪄먹는 건강한 야채찜입니다:-...,[현미라이스페이퍼] 현미라이스페이퍼 14장 [속재료] 국산 두부 1/2모| 친환경 ...,"[현미라이스페이퍼, 국산 두부, 친환경 표고버섯, 친환경 빨강파프리카, 친환경 노랑...",364.387618
93880,6947705,매콤한 야채 치즈 가리비 구이,야채치즈가리비구이,1271,0,34,굽기,술안주,해물류,메인반찬,가리비 철이라고 막내가 보내 왔네요 맛있게 쪄서도 먹고 야채 치즈구이도 해봤습니다 ...,[재료] 가리비 8개| 다진양파 1+1/2큰술| 다진당근 1+1/2큰술| 다진토마토...,"[가리비, 다진양파, 다진당근, 다진토마토, 치즈, 톡톡채소대파, 톡톡채소청양고추]",354.881713
22274,6837314,개운한 코다리매운탕,코다리매운탕,7833,4,103,끓이기,일상,건어물류,국/탕,오늘 레시피 포스팅에서는 깔끔하고 얼큰한 국물요리 코다리 매운탕을 만들어볼거에요. ...,[재료] 자세한 재료는 본문내용에서| 확인하실 수 있습니다.,"[자세한 재료는, 확인하실 수]",281.060904


In [73]:
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in euc_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  {'콩나물', '고추', '홍합', '미나리', '무', '해물탕 양념', '파', '바지락', '아귀'}


[['먹다남은 야채수제비육수', '수제비반죽', '체다치즈'],
 ['비빔면', '새러드 채소', '토마토'],
 ['현미라이스페이퍼',
  '국산 두부',
  '친환경 표고버섯',
  '친환경 빨강파프리카',
  '친환경 노랑파프리카',
  '유기농 당근 작은것',
  '유정란',
  '국산감자전분가루    찹쌀가루',
  '죽염'],
 ['가리비', '다진양파', '다진당근', '다진토마토', '치즈', '톡톡채소대파', '톡톡채소청양고추'],
 ['자세한 재료는', '확인하실 수']]

3. 맨하탄 유사도 

In [74]:
#3. 맨하탄 유사도 
mealkit_index = len(w2v_man)-1

man_mealkit_df = wv_mealkit(mealkit_index, w2v_man, 5)
man_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
29687,6851213,[해외자취Cook.feel通]157. 치즈야채수제비 레시피,치즈야채수제비,450,0,3,끓이기,초스피드,밀가루,면/만두,전날 한 야채수제비가 2인분이라 수제비 반죽을 뜯어 넣기 전에 1인분을 따로 덜어놨...,[재료] 먹다남은 야채수제비육수 2국자| 수제비반죽 1주먹| 체다치즈 1장,"[먹다남은 야채수제비육수, 수제비반죽, 체다치즈]",372.936123
38594,6867336,야채가 들어간 비빔면,야채비빔면,704,2,5,비빔,초스피드,채소류,면/만두,봄이 오니까 자꾸 시원한게 생각이 나시죠? 저두요. . . 그래서 비빔면 삶아 샐러...,[재료] 비빔면 1봉| 새러드 채소 약간| 토마토 1/6개,"[비빔면, 새러드 채소, 토마토]",366.359289
95027,6949241,[친환경자연쿡]현미라이스페이퍼야채찜,현미라이스페이퍼야채찜,790,0,9,찜,영양식,기타,메인반찬,현미라이스페이퍼로 두부와 야채를 잘게 다져 돌돌 말아 쪄먹는 건강한 야채찜입니다:-...,[현미라이스페이퍼] 현미라이스페이퍼 14장 [속재료] 국산 두부 1/2모| 친환경 ...,"[현미라이스페이퍼, 국산 두부, 친환경 표고버섯, 친환경 빨강파프리카, 친환경 노랑...",364.387618
93880,6947705,매콤한 야채 치즈 가리비 구이,야채치즈가리비구이,1271,0,34,굽기,술안주,해물류,메인반찬,가리비 철이라고 막내가 보내 왔네요 맛있게 쪄서도 먹고 야채 치즈구이도 해봤습니다 ...,[재료] 가리비 8개| 다진양파 1+1/2큰술| 다진당근 1+1/2큰술| 다진토마토...,"[가리비, 다진양파, 다진당근, 다진토마토, 치즈, 톡톡채소대파, 톡톡채소청양고추]",354.881713
22274,6837314,개운한 코다리매운탕,코다리매운탕,7833,4,103,끓이기,일상,건어물류,국/탕,오늘 레시피 포스팅에서는 깔끔하고 얼큰한 국물요리 코다리 매운탕을 만들어볼거에요. ...,[재료] 자세한 재료는 본문내용에서| 확인하실 수 있습니다.,"[자세한 재료는, 확인하실 수]",281.060904


In [75]:
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in man_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  {'콩나물', '고추', '홍합', '미나리', '무', '해물탕 양념', '파', '바지락', '아귀'}


[['먹다남은 야채수제비육수', '수제비반죽', '체다치즈'],
 ['비빔면', '새러드 채소', '토마토'],
 ['현미라이스페이퍼',
  '국산 두부',
  '친환경 표고버섯',
  '친환경 빨강파프리카',
  '친환경 노랑파프리카',
  '유기농 당근 작은것',
  '유정란',
  '국산감자전분가루    찹쌀가루',
  '죽염'],
 ['가리비', '다진양파', '다진당근', '다진토마토', '치즈', '톡톡채소대파', '톡톡채소청양고추'],
 ['자세한 재료는', '확인하실 수']]

# tfidf 기반

In [76]:
# product = product_df.상품.values
# product_list = np.array2string(product).split(' ')

# category = product_df.상세카테고리.values

# recipe_df = pd.DataFrame()
# for word in product_list:
#   recipe = ckg.loc[ckg['요리명']==word]
#   recipe_df = pd.concat([recipe_df, recipe])
# recipe_df

In [77]:
#구성정보를 공백기준 하나의 string으로 변경
str_list = product_df.구성정보_전처리.values.tolist()
ingredient = ' '.join(s for s in str_list).split(',')
ingredient = ' '.join(ingredient)
#print(ingredient)

#요리재료를 공백기준 하나의 string으로 변경
recipe_df['tfidf']  = recipe_df.요리재료_전처리.apply(lambda x : (' ').join(x))
docs = recipe_df.tfidf.values

#ndarray의 가장 마지막에 input값 연결
docs = np.append(docs, ingredient)
#print(docs, type(docs))

#tfidf 적용
tfid = TfidfVectorizer()
tfvect = tfid.fit_transform(docs)
tfvect_df = pd.DataFrame(tfvect.toarray(), columns = sorted(tfid.vocabulary_))

# 1. 코사인유사도
tf_sim=cosine_similarity(tfvect_df, tfvect_df)
# 2. 유클리디언 유사도
tf_euc=euclidean_distances(tfvect_df, tfvect_df)
# 3. 맨하탄 유사도 
tf_man=manhattan_distances(tfvect_df, tfvect_df)

In [78]:
#최종 알고리즘
def tf_mealkit(mealkit_index,sim,top_n):
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]

    # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
    similar_indexes=sim_sorted_ind[mealkit_index,1:(top_n+1)]
    recipe = recipe_df.iloc[similar_indexes].sort_values(by='weighted_clipping',ascending=False).drop('tfidf', axis=1)
    
    return recipe

1. 코사인유사도

In [79]:
# 1. 코사인유사도
mealkit_index = -1
matrix = tf_sim

sim_mealkit_df = tf_mealkit(mealkit_index, matrix, 5)
sim_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
28913,6849961,[콩나물조개매운탕 만들기],콩나물조개매운탕,2336,1,36,끓이기,일상,채소류,국/탕,얼큰하고 시원한맛 http://blog.naver.com/ajok10,[재료] 바지락 1봉지| 다시마 2장| 물 5+1/2컵,"[바지락, 다시마, 물]",333.419311
80771,6930106,개운하고 맛있는 맑은아귀탕 식초가 tip,맑은아귀탕,2914,0,22,끓이기,일상,기타,국/탕,개운한 국물맛 일품인 맑은아귀탕 오랜만에 끓여요,[재료] 아귀 1마리| 무 50g| 콩나물 200g| 미나리 1/2단| 대파 1/2...,"[아귀, 무, 콩나물, 미나리, 대파, 다진마늘, 국간장, 식초, 소금]",320.356340
28607,6848317,시원한 국물맛이 일품 아귀맑은탕,아귀탕,7536,4,54,끓이기,일상,해물류,국/탕,날이 더워져도 시원한 국물이 생각 날 때가 있어요 그런날 먹기에 딱 좋은 아귀맑은탕...,[재료] 아귀 1마리| 콩나물 50g| 미나리 20g [양념] 다진마늘 1/2T| ...,"[아귀, 콩나물, 미나리, 다진마늘, 다진파, 국간장, 소금, 홍고추]",265.926729
71576,6917383,간단요리...아귀수육,아귀수육,8706,0,34,삶기,술안주,기타,기타,여름에 깔끔해서 만들어봤어요,[재료] 아귀 1마리| 콩나물 100g| 미나리 100g| 간장 2T| 와사비 1T...,"[아귀, 콩나물, 미나리, 간장, 와사비, 청양고추]",246.806717
5664,2662422,아귀로 3가지 요리 즐기기,아귀,19623,11,102,끓이기,손님접대,해물류,메인반찬,아귀로 만든 찜 비빔밥 매운탕 요리입니다. 그 어떤 찌개나 국보다 찜요리가 제일...,[아귀찜 재료] 아귀 1kg| 콩나물 500g| 미나리 200g| 대파 2대| 참기...,"[아귀, 콩나물, 미나리, 대파, 참기름, 밥, 미나리, 아귀, 미나리, 콩나물, ...",216.834393


In [80]:
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in sim_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  아귀 바지락 홍합 콩나물 미나리 파 고추 무 해물탕 양념


[['다시마'],
 ['무', '미나리', '대파', '다진마늘', '국간장', '식초', '소금'],
 ['콩나물', '다진마늘', '다진파', '국간장', '소금', '홍고추'],
 ['콩나물', '간장', '와사비', '청양고추'],
 ['콩나물',
  '대파',
  '참기름',
  '밥',
  '아귀',
  '콩나물',
  '마늘',
  '청양고추',
  '육수',
  '고추가루',
  '고추장',
  '굴소스',
  '간장',
  '청주',
  '다진 마늘',
  '후추',
  '고추냉이',
  '설탕',
  '찹쌀가루',
  '남은아귀찜양념',
  '참기름또는들기름',
  '고춧가루',
  '국간장',
  '소금']]

2. 유클리디언 유사도

In [81]:
# 2. 유클리디언 유사도
mealkit_index = -1
matrix = tf_euc

euc_mealkit_df = tf_mealkit(mealkit_index, matrix, 5)
euc_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
24893,6841883,쏘세지야채볶음 만들기,소세지야채볶음,62184,39,1597,볶음,일상,가공식품류,메인반찬,비엔나 쏘세지와 야채를 넣어 만든 쏘세지야채볶음이에요.,[재료] 비엔나 소세지 30개| 양파 1/2개| 당근 1/3개| 브로콜리 약간 [양...,"[비엔나 소세지, 양파, 당근, 브로콜리, 케첩, 굴소스, 올리고당, 핫소스, 설탕]",1382.606942
24487,6841274,매콤한 소세지야채볶음,소세지야채볶음,24608,54,896,볶음,일상,가공식품류,밑반찬,냉장고 뒤적뒤적 해답은? 매콤한 소세지야채볶음 되시겠어요. 간단한 메뉴이지만 소스...,[재료] 비엔나소세지 110g| 빨강파프리카 1/2개| 노랑파프리카 1/2개| 청양...,"[비엔나소세지, 빨강파프리카, 노랑파프리카, 청양고추, 통깨가루, 고추장, 올리고당...",716.249895
24402,6841154,새우야채전 맛나는 간단반찬으로 제격!,새우야채전,11012,11,332,부침,일상,해물류,밑반찬,오동통한 새우살이 씹히는 맛이 좋고 다양한 채소들로 영양 가득 맛도 좋은 간단반찬이에요,[재료] 새우| 애호박| 당근| 양파| 계란| 부침가루| 후추| 소금,"[새우, 애호박, 당근, 양파, 계란, 부침가루, 후추, 소금]",361.274692
24299,6840995,소시지 야채 프리타타,소시지야채프리타타,3721,3,99,굽기,일상,달걀/유제품,양식,냉장고 속 야채들을 우아하게 처리할 수 있는 소시지 야채 프리타타를 만들었어요. 이...,[재료] 소시지 2개| 양파 1/2개| 파프리카 1/2개| 표고버섯 2개| 양송이버...,"[소시지, 양파, 파프리카, 표고버섯, 양송이버섯, 시금치, 계란, 물, 소금, 올...",323.032959
24203,6840840,야채 김 계란말이 맛있고 부드럽게 만드는법~,야채계란말이,5438,4,31,부침,일상,달걀/유제품,밑반찬,안녕하세요~!~ 어젯밤부터 몸이 으슬으슬 춥더니 목이 붓고 온몸이 쑤시고 아프네요~...,[재료] 실키란(계란) 5개| 치커리 10g| 당근 20g| 김 1장 [양념] 액젓...,"[실키란, 치커리, 당근, 김, 액젓, 마요네즈, 감자전분, 피자치즈, 식용유]",282.917652


In [82]:
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in euc_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  아귀 바지락 홍합 콩나물 미나리 파 고추 무 해물탕 양념


[['비엔나 소세지', '양파', '당근', '브로콜리', '케첩', '굴소스', '올리고당', '핫소스', '설탕'],
 ['비엔나소세지',
  '빨강파프리카',
  '노랑파프리카',
  '청양고추',
  '통깨가루',
  '고추장',
  '올리고당',
  '간장',
  '후추',
  '다진 마늘',
  '맛술',
  '참기름',
  '케첩'],
 ['새우', '애호박', '당근', '양파', '계란', '부침가루', '후추', '소금'],
 ['소시지', '양파', '파프리카', '표고버섯', '양송이버섯', '시금치', '계란', '소금', '올리브유', '후추'],
 ['실키란', '치커리', '당근', '김', '액젓', '마요네즈', '감자전분', '피자치즈', '식용유']]

3. 맨하탄 유사도 

In [83]:
# 3. 맨하탄 유사도 
mealkit_index = -1
matrix = tf_man

man_mealkit_df = tf_mealkit(mealkit_index, matrix, 5)
man_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
53245,6890596,아귀 냉콩국 수제비(초간편).콩국수 아님 주의!,아귀냉콩국수제비,1726,0,13,기타,초스피드,해물류,면/만두,초스피드한데 엄청 든든한 한 끼?! 여름에 시원한 콩국수 많이 드시죠~ 비린향으로 ...,[아귀 비린내 제거] 해동한 순살 아귀 110g| 물 많이| 녹차 티백 1개| 청주...,"[해동한 순살, 물, 녹차 티백, 청주, 순두부, 우유, 캔, 삶아서 물기를 뺀, ...",342.808366
49585,6884821,베이컨 야채 리조또,베이컨야채리조또,2676,0,21,볶음,일상,쌀,밥/죽/떡,최근 외식을 하며 리조또를 처음으로 먹어봤는데 정말 매력있는 요리 같아서 기억에 남...,[재료] 쌀 300g| 베이컨 3줄| 양송이버섯 3개| 새송이버섯 1개| 올리브오일...,"[쌀, 베이컨, 양송이버섯, 새송이버섯, 올리브오일, 양파, 마늘, 브로콜리, 컬리...",324.615162
53464,6890928,오징어 야채 무침,오징어야채무침,3368,0,58,무침,일상,해물류,밑반찬,오징어 무침을 만들었습니다 오이의 아삭하고 오징어의 쫄깃함이 잘 어우려지는 맛입니다,[재료] 오징어 1마리| 맛술 1큰술| 오이 1개| 소금 1작은술| 물엿 1큰술| ...,"[오징어, 맛술, 오이, 소금, 물엿, 당근, 양파, 피망, 대파, 고추장, 고추가...",319.507575
423,895284,캔 참치의 양을 왕창 늘려주는 엄마표 야채참치,야채참치,27330,23,232,볶음,일상,채소류,밑반찬,저 어릴때 주로 도시락 반찬으로 종종 싸주셨던 야채참치를 다시 한번 또 재현해서 채...,[재료] 참치 통조림 150g| 감자(중) 1개| 양파(중) 1/2개| 당근(중) ...,"[참치, 감자, 양파, 당근, 옥수수 통조림, 대파 파란 부분, 고추장, 케첩, 간...",282.385667
15916,6765284,우럭 매운탕,우럭매운탕,18226,9,109,끓이기,일상,해물류,국/탕,얼큰하고 시원한 우럭 매운탕을 만들어 보겠습니다.,[재료] 우럭회 1팩| 황태 멸치 육수 1리터| 무 나박썬 것 1줌| 호박 썬 것 ...,"[우럭회, 황태 멸치 육수, 무 나박썬 것, 호박 썬 것, 콩나물, 양파, 두부, ...",225.949938


In [84]:
print('ingredient = ', ingredient)
recipe_ingredient = []
for items in man_mealkit_df.요리재료_전처리:
  for item in items:
    if item in ingredient:
      items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

ingredient =  아귀 바지락 홍합 콩나물 미나리 파 고추 무 해물탕 양념


[['해동한 순살',
  '녹차 티백',
  '청주',
  '순두부',
  '우유',
  '캔',
  '삶아서 물기를 뺀',
  '깨',
  '구운 통아몬드',
  '시판용 만두피',
  '파마산 치즈',
  '찬물',
  '과자 식감의 누룽지'],
 ['쌀',
  '베이컨',
  '양송이버섯',
  '새송이버섯',
  '올리브오일',
  '양파',
  '마늘',
  '브로콜리',
  '컬리플라워',
  '완두콩',
  '치킨스톡',
  '파마산',
  '모짜렐라',
  '생크림',
  '후추 허브',
  '그라나파다노  파마산  파르미지아노 레지아노',
  '파슬리',
  '아이순  어린잎  루꼴라'],
 ['오징어',
  '맛술',
  '오이',
  '소금',
  '물엿',
  '당근',
  '양파',
  '피망',
  '대파',
  '고추장',
  '고추가루',
  '다진마늘',
  '진간장',
  '식초',
  '생강청',
  '매실청',
  '후추',
  '설탕',
  '참치액',
  '참기름',
  '깨소금'],
 ['참치',
  '감자',
  '양파',
  '당근',
  '옥수수 통조림',
  '대파 파란 부분',
  '고추장',
  '케첩',
  '간장',
  '맛술',
  '다진 마늘',
  '설탕',
  '물엿',
  '생강가루',
  '후춧가루',
  '통깨',
  '참기름'],
 ['우럭회',
  '황태 멸치 육수',
  '무 나박썬 것',
  '호박 썬 것',
  '양파',
  '두부',
  '쑥갓',
  '대파',
  '홍고추',
  '풋고추',
  '양념장',
  '고추가루',
  '생강가루',
  '통후추 간 것',
  '다진마늘    큰술',
  '멸치',
  '황태머리',
  '마른 다시마',
  '양파 작은',
  '대파 흰 부분',
  '청주',
  '소금',
  '설탕']]